# Apple IPv6 time server issues

There have [been reports of one of the IPv6 addresses](https://community.ziggo.nl/t5/user/viewprofilepage/user-id/128998) of `time.apple.com` having issues:


For one, Apple runs their own time servers, so their clients are configured to query `time.apple.com` 

In IPv4, the addresses are:

```shell
$ dig a time.apple.com +short
17.253.52.253
17.253.52.125
17.253.14.251
```

And IPv6:

```shell
$dig a time.apple.com +short
2a01:b740:a30:4000::1f2
2a01:b740:a30:3000::1f2
2a01:b740:a30:4000::1e2
```



 TO investigate that, we use ~ 4k vantage points  from [Ripe Atlas](https://atlas.ripe.net)
 
 Each probe is configured to send 3 NTP packets to `2a01:b740:a30:3000::1f2`
 
 https://atlas.ripe.net/measurements/86426966/overview

In [1]:
import duckdb as db

import numpy as np
con = db.connect('apple')

In [4]:
#let's get all Atlas probes metadata first
con.load_extension("shellfs")
probes_data = con.sql(
    """
        CREATE OR REPLACE TABLE probes AS (
            SELECT * 
            FROM read_json("curl -s https://ftp.ripe.net/ripe/atlas/probes/archive/2025/01/20250127.json.bz2 | bunzip2 -d | jq '.objects' |")
        )
    """
)

con.sql("SELECT * FROM probes LIMIT 5")

┌───────┬────────────────┬──────────────────────┬────────┬───┬──────────┬──────────────────────┬─────────────┐
│  id   │   address_v4   │      address_v6      │ asn_v4 │ … │   day    │        probe         │ status_name │
│ int64 │    varchar     │       varchar        │ int64  │   │ varchar  │       varchar        │   varchar   │
├───────┼────────────────┼──────────────────────┼────────┼───┼──────────┼──────────────────────┼─────────────┤
│     1 │ 45.138.229.91  │ 2a10:3781:e22:1:22…  │ 206238 │ … │ 20250127 │ https://atlas.ripe…  │ Connected   │
│     2 │ NULL           │ NULL                 │   1136 │ … │ 20250127 │ https://atlas.ripe…  │ Abandoned   │
│     3 │ 77.174.76.85   │ 2a02:a467:f500:1:2…  │   1136 │ … │ 20250127 │ https://atlas.ripe…  │ Connected   │
│     4 │ 83.163.50.165  │ 2001:980:57a4:1:22…  │   3265 │ … │ 20250127 │ https://atlas.ripe…  │ Abandoned   │
│     5 │ 83.163.239.181 │ 2001:981:602b:1:22…  │   3265 │ … │ 20250127 │ https://atlas.ripe…  │ Abandoned   │
├

In [8]:
# Create the table 'results'  to store results from measurements
con.execute('''
CREATE or replace  TABLE results (
    id INTEGER,
    ntp_offset FLOAT,
    n_res INTEGER
)
''')

In [12]:
# now let's get the measurements
import requests
import json
url='https://atlas.ripe.net/api/v2/measurements/86426966/results/?format=json'

response = requests.get(url)

# Parse the JSON content
data = json.loads(response.text)

In [14]:
#now let's get three variables from the data, there are more
#probe_id, offset, and nubmer of responses


#now let' s parse these results and add it to 'res', a list that stores it

timeout_all=[]
res_all=[]
for k in data:
    probe=k['prb_id']
    if 'result' in k and len(k['result'])>0:
        offset=[]
        for x in k['result']:
            
            if 'offset' not in x:
                timeout_all.append(k)
            else:

                offset.append(x['offset'])
                
    #print(probe,np.average(offset),len(offset))
    res_all.append(str(probe)+","+ str(np.average(offset))+","  +str(len(offset)))

/usr/lib/python3/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/lib/python3/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [16]:
# Split the elements by ',' and convert the data types
data_tuples = [tuple(int(x) if i != 1 else float(x) for i, x in enumerate(item.split(','))) for item in res_all]

# Insert data into the table
con.executemany('INSERT INTO results VALUES (?, ?, ?)', data_tuples)

In [18]:
#results

# if n_res != 3, it means that the VP cannot reach the ntp server at the time
# for simplicity, let's get all the probes which could not all all reach the ntp server


x=con.sql("select n_res, count(distinct  id) as n_probes from results group by n_res ")
print(x.df())



   n_res  n_probes
0      0       388
1      3      3964
2      2        58
3      1        33


In [20]:
# Create a new table with the filtered rows
#only keep the probes that could not reach the time server
con.execute('''
CREATE or replace TABLE intersection_probes AS
SELECT m.*
FROM probes m
INNER JOIN results p ON m.id = p.id 
WHERE p.n_res = 0

''')

In [22]:
x=con.sql('select count(1) from intersection_probes')
print(x.df())

   count(1)
0       388


In [35]:
# now, let's see where they are from
x=con.sql('''
select asn_v6, country_code, count(distinct id) 
as total from intersection_probes 
where status=1
group by  asn_v6, country_code

order by total desc
''')

print(x.df().to_string())

       asn_v6 country_code  total
0     33915.0           NL     10
1         NaN           US     10
2         NaN           RU      7
3         NaN           GB      6
4         NaN           FR      6
5      8151.0           MX      5
6      6848.0           BE      5
7     12322.0           FR      4
8         NaN           CA      4
9     56478.0           GB      4
10        NaN           DE      4
11     8473.0           SE      4
12     6939.0           RU      4
13        NaN           SE      4
14     4764.0           AU      3
15        NaN           AT      3
16        NaN           JP      3
17        NaN           FI      3
18        NaN           IT      3
19        NaN           PL      3
20     3320.0           DE      3
21        NaN           NL      3
22     9595.0           JP      3
23     2860.0           PT      3
24      852.0           CA      2
25        NaN           IE      2
26   211450.0           MK      2
27     2027.0           FR      2
28        NaN 

In [37]:
# let's get some sample results

x=con.sql('select * from intersection_probes limit 10')
print(x.df())

     id       address_v4                              address_v6  asn_v4  \
0   416     91.194.69.13       2001:678:150:3:220:4aff:fec6:cc8e   59617   
1   677     95.42.140.70  fd32:200f:e857:d20c:220:4aff:fec5:5450    8866   
2   867    82.148.227.13        2a00:1b98:1:0:220:4aff:fec7:ae01   25376   
3   890    80.92.113.113                 2a02:28e8:d0d0:15::dead   34347   
4  1114  171.102.153.241    2001:67c:1232:144:220:4aff:fec7:b0b6    7470   
5  1127     66.42.191.76  2600:2b00:8a4f:6ff9:220:4aff:fec7:b079    6181   
6  2263     93.186.46.61  fdba:f13d:7b7a:fdba:220:4aff:fee0:252a    3213   
7   964    91.215.148.67     2a03:5380:0:1890:220:4aff:fec7:afcf   34612   
8  2047     84.24.241.71   2001:1c00:e8e:c100:220:4aff:fee0:244c   33915   
9  3450    27.94.196.126   fd06:bdff:cf8:741e:280:a3ff:fe91:442d    2516   

    asn_v6         prefix_v4           prefix_v6  is_anchor  is_public  \
0  59617.0    91.194.69.0/24   2001:678:150::/48      False       True   
1      NaN    9